In [7]:
# Import packages
import pandas as pd
from sqlalchemy import create_engine
import json
import os

pd.set_option('display.max_rows', 200)

In [8]:
%load_ext dotenv
%dotenv ../.env

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [9]:
con = create_engine(os.getenv('BUILD_ENGINE').replace('localhost', os.getenv('IP')))

In [10]:
# Build funtion to get the domains associated with a specific field
def get_domain(field, domain_path):
    with open(domain_path, "r") as domain:
        domain = json.load(domain)
    records = [i for i in domain['fields'] if i['field'] == field]
    if len(records) == 0: 
        print(f'cannot find field: {field} in domain table')
    else: 
        return records[0]['domain']

In [11]:
# Example
proxcode_domain = get_domain('proxcode', 'pluto_domains.json')
print(proxcode_domain)

{'0': 'Not Available', '1': 'Detached', '2': 'Semi-Attached', '3': 'Attached'}


In [12]:
# Example
proxcode_domain.keys()

dict_keys(['0', '1', '2', '3'])

In [24]:
# Specify latest PLUTO version
pluto = 'pluto'
# Specify previous PLUTO version or the one you want to compare to
plutoprev = 'dcp_pluto'

### Value check for the thirteen fields with expected domain values

##### includes summary statistics of unique value comparisions for the two PLUTO version, and lists of values that do not agree w/ domain list

### Zoning districts

In [25]:
sql = f'''
    WITH newcount as (
    WITH zdall as (
    SELECT zonedist1 as zonedist FROM {pluto}
    UNION ALL
    SELECT zonedist2 as zonedist FROM {pluto}
    UNION ALL
    SELECT zonedist3 as zonedist FROM {pluto}
    UNION ALL
    SELECT zonedist4 as zonedist FROM {pluto})
    SELECT zonedist, COUNT(*) 
    FROM zdall
    GROUP BY zonedist),
    prevcount as (
    WITH zdall as (
    SELECT zonedist1 as zonedist FROM {plutoprev}
    UNION ALL
    SELECT zonedist2 as zonedist FROM {plutoprev}
    UNION ALL
    SELECT zonedist3 as zonedist FROM {plutoprev}
    UNION ALL
    SELECT zonedist4 as zonedist FROM {plutoprev})
    SELECT zonedist, COUNT(*) 
    FROM zdall
    GROUP BY zonedist)
    SELECT a.zonedist, b.zonedist as zonedistprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.zonedist = b.zonedist
    ORDER BY diff DESC;
'''

In [26]:
df = pd.read_sql(sql=sql, con=con)

In [27]:
df[df['diff'] != 0]

,zonedist,zonedistprev,countnew,countprev,diff
0,None,None,2562457.0,NaN,NaN
1,PUBLIC PLACE,None,15.0,NaN,NaN
2,PLAYGROUND,None,90.0,NaN,NaN
3,BALL FIELD,None,5.0,NaN,NaN
4,None,None,NaN,2560881.0,NaN
5,C1-9A,C1-9A,136.0,134.0,2.0
6,C5-5,C5-5,295.0,294.0,1.0
95,M1-6/R10,M1-6/R10,75.0,76.0,-1.0
96,R2A,R2A,38220.0,38221.0,-1.0
97,M1-3/R7X,M1-3/R7X,48.0,49.0,-1.0


### Commercial overlay

In [28]:
sql = f'''
    WITH newcount as (
    WITH olall as (
    SELECT overlay1 as overlay FROM {pluto}
    UNION ALL
    SELECT overlay2 as overlay FROM {pluto}
    )
    SELECT overlay, COUNT(*) 
    FROM olall
    GROUP BY overlay),
    prevcount as (
    WITH olall as (
    SELECT overlay1 as overlay FROM {plutoprev}
    UNION ALL
    SELECT overlay2 as overlay FROM {plutoprev})
    SELECT overlay, COUNT(*) 
    FROM olall
    GROUP BY overlay)
    SELECT a.overlay, b.overlay as overlayprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.overlay = b.overlay
    ORDER BY a.overlay;
'''

In [29]:
df = pd.read_sql(sql=sql, con=con)

In [30]:
df

,overlay,overlayprev,countnew,countprev,diff
0,C1-1,C1-1,1586.0,1587.0,-1.0
1,C1-2,C1-2,11029.0,11031.0,-2.0
2,C1-3,C1-3,13779.0,13790.0,-11.0
3,C1-4,C1-4,8092.0,8097.0,-5.0
4,C1-5,C1-5,2538.0,2547.0,-9.0
5,C2-1,C2-1,1684.0,1684.0,0.0
6,C2-2,C2-2,6026.0,6023.0,3.0
7,C2-3,C2-3,11961.0,11952.0,9.0
8,C2-4,C2-4,16713.0,16727.0,-14.0
9,C2-5,C2-5,1522.0,1523.0,-1.0


### Special purpose district

In [31]:
sql = f'''
    WITH newcount as (
    WITH sdall as (
    SELECT spdist1 as spdist FROM {pluto}
    UNION ALL
    SELECT spdist2 as spdist FROM {pluto}
    UNION ALL
    SELECT spdist3 as spdist FROM {pluto}
    )
    SELECT spdist, COUNT(*) 
    FROM sdall
    GROUP BY spdist),
    prevcount as (
     WITH sdall as (
    SELECT spdist1 as spdist FROM {plutoprev}
    UNION ALL
    SELECT spdist2 as spdist FROM {plutoprev}
    UNION ALL
    SELECT spdist3 as spdist FROM {plutoprev}
    )
    SELECT spdist, COUNT(*) 
    FROM sdall
    GROUP BY spdist)
    SELECT a.spdist, b.spdist as spdistprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.spdist = b.spdist
    ORDER BY diff DESC;
'''

In [32]:
df = pd.read_sql(sql=sql, con=con)

In [33]:
df[df['diff'] != 0]

,spdist,spdistprev,countnew,countprev,diff
0,None,None,NaN,2476583.0,NaN
1,None,None,2476554.0,NaN,NaN
2,SRD,SRD,44594.0,43889.0,705.0
3,NA-1,NA-1,2447.0,2408.0,39.0
4,HS,HS,6195.0,6165.0,30.0
5,BR,BR,9716.0,9706.0,10.0
6,LM,LM,567.0,559.0,8.0
7,NA-2,NA-2,989.0,981.0,8.0
8,J,J,474.0,469.0,5.0
9,OP,OP,13448.0,13443.0,5.0


In [34]:
#Check spdist domain
unique_spdist = df.spdist.tolist()
spdist_domain = list(get_domain('spdist1', 'pluto_domains.json').keys())

In [35]:
# in domain not in pluto:
[i for i in spdist_domain if i not in unique_spdist]

['MX-3']

In [36]:
# in pluto not in domain:
[i for i in unique_spdist if i not in spdist_domain]

[None, None, 'MX-20', 'MX-16/EC-5', 'BSC', 'SW', 'MX-16/EC-6']

### Limited height

In [37]:
sql = f'''
    WITH newcount as (
    SELECT ltdheight, count(*)
    FROM {pluto}
    GROUP BY ltdheight),
    prevcount as (
    SELECT ltdheight, count(*)
    FROM {plutoprev}
    GROUP BY ltdheight)
    SELECT a.ltdheight, b.ltdheight as ltdheightprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.ltdheight = b.ltdheight
    ORDER BY a.ltdheight;
'''

In [38]:
df = pd.read_sql(sql=sql, con=con)

In [39]:
df

,ltdheight,ltdheightprev,countnew,countprev,diff
0,LH-1,LH-1,2248.0,2239.0,9.0
1,LH-1A,LH-1A,797.0,797.0,0.0
2,None,None,856666.0,NaN,NaN
3,None,None,NaN,856423.0,NaN


In [40]:
#Check ltdheight domain
unique_ltdheight = df.ltdheight.tolist()
ltdheight_domain = list(get_domain('ltdheight', 'pluto_domains.json').keys())

In [41]:
# in domain not in pluto:
[i for i in ltdheight_domain if i not in unique_ltdheight]

['LH-2', 'LH-3']

In [42]:
# in pluto not in domain:
[i for i in unique_ltdheight if i not in ltdheight_domain]

[None, None]

### Building class

In [43]:
sql = f'''
    WITH newcount as (
    SELECT bldgclass, count(*)
    FROM {pluto}
    GROUP BY bldgclass),
    prevcount as (
    SELECT bldgclass, count(*)
    FROM {plutoprev}
    GROUP BY bldgclass)
    SELECT a.bldgclass, b.bldgclass as bldgclassprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.bldgclass = b.bldgclass
    ORDER BY diff DESC;
'''

In [44]:
df = pd.read_sql(sql=sql, con=con)

In [45]:
df[df['diff'] != 0]

,bldgclass,bldgclassprev,countnew,countprev,diff
0,None,None,NaN,473.0,NaN
1,None,RP,NaN,31.0,NaN
2,None,None,986.0,NaN,NaN
3,None,RT,NaN,8.0,NaN
4,V1,V1,6854.0,6671.0,183.0
5,G7,G7,5380.0,5290.0,90.0
6,Z7,Z7,176.0,109.0,67.0
7,D7,D7,2181.0,2119.0,62.0
8,C0,C0,73056.0,72995.0,61.0
9,D6,D6,1155.0,1096.0,59.0


In [46]:
#Check bldgclass domain
unique_bldgclass = df.bldgclass.tolist()
bldgclass_domain = list(get_domain('bldgclass', 'pluto_domains.json').keys())

In [47]:
# in domain not in pluto:
[i for i in bldgclass_domain if i not in unique_bldgclass]

['A',
 'B',
 'C',
 'D',
 'E',
 'E3',
 'E4',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'R7',
 'RP',
 'RT',
 'S',
 'T',
 'U',
 'U3',
 'V',
 'W',
 'Y',
 'Z',
 'Z6']

In [48]:
# in pluto not in domain:
[i for i in unique_bldgclass if i not in bldgclass_domain]

[None, None, None, None]

### Land use

In [49]:
sql = f'''
    WITH newcount as (
    SELECT landuse, count(*)
    FROM {pluto}
    GROUP BY landuse),
    prevcount as (
    SELECT landuse, count(*)
    FROM {plutoprev}
    GROUP BY landuse)
    SELECT a.landuse, b.landuse as landuseprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.landuse = b.landuse
    ORDER BY a.landuse;
'''

In [50]:
df = pd.read_sql(sql=sql, con=con)

In [51]:
df[df['diff'] != 0]

,landuse,landuseprev,countnew,countprev,diff
0,01,01,565392.0,565356.0,36.0
1,02,02,131230.0,131378.0,-148.0
2,03,03,12555.0,12846.0,-291.0
3,04,04,54618.0,54508.0,110.0
4,05,05,21411.0,21482.0,-71.0
5,06,06,10514.0,10497.0,17.0
6,07,07,6314.0,6335.0,-21.0
7,08,08,12204.0,12177.0,27.0
8,09,09,3963.0,4017.0,-54.0
9,10,10,10212.0,10224.0,-12.0


In [52]:
#Check landuse domain
unique_landuse = df.landuse.tolist()
landuse_domain = list(get_domain('landuse', 'pluto_domains.json').keys())

In [53]:
# in domain not in pluto:
[i for i in landuse_domain if i not in unique_landuse]

[]

In [54]:
# in pluto not in domain:
[i for i in unique_landuse if i not in landuse_domain]

[None, None]

### Owner type

In [55]:
sql = f'''
    WITH newcount as (
    SELECT ownertype, count(*)
    FROM {pluto}
    GROUP BY ownertype),
    prevcount as (
    SELECT ownertype, count(*)
    FROM {plutoprev}
    GROUP BY ownertype)
    SELECT a.ownertype, b.ownertype as ownertypeprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.ownertype = b.ownertype
    ORDER BY a.ownertype;
'''

In [56]:
df = pd.read_sql(sql=sql, con=con)

In [57]:
df

,ownertype,ownertypeprev,countnew,countprev,diff
0,C,C,11676.0,11654.0,22.0
1,M,M,80.0,80.0,0.0
2,O,O,1374.0,1373.0,1.0
3,P,P,523.0,522.0,1.0
4,X,X,17333.0,17414.0,-81.0
5,None,None,828725.0,NaN,NaN
6,None,None,NaN,828416.0,NaN


In [58]:
#Check ownertype domain
unique_ownertype = df.ownertype.tolist()
ownertype_domain = list(get_domain('ownertype', 'pluto_domains.json').keys())

In [59]:
# in domain not in pluto:
[i for i in ownertype_domain if i not in unique_ownertype]

['NaN']

In [60]:
# in pluto not in domain:
[i for i in unique_ownertype if i not in ownertype_domain]

[None, None]

### Area source

In [61]:
sql = f'''
    WITH newcount as (
    SELECT areasource, count(*)
    FROM {pluto}
    GROUP BY areasource),
    prevcount as (
    SELECT areasource, count(*)
    FROM {plutoprev}
    GROUP BY areasource)
    SELECT a.areasource, b.areasource as areasourceprev, a.count as countnew, b.count as countprev, a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.areasource = b.areasource
    ORDER BY a.areasource;
'''

In [62]:
df = pd.read_sql(sql=sql, con=con)

In [63]:
df

,areasource,areasourceprev,countnew,countprev,diff
0,0,0,338.0,136.0,202.0
1,2,2,812678.0,813263.0,-585.0
2,4,4,422.0,182.0,240.0
3,5,5,13.0,63.0,-50.0
4,7,7,45274.0,45342.0,-68.0
5,None,None,NaN,473.0,NaN
6,None,None,986.0,NaN,NaN


In [64]:
#Check areasource domain
unique_areasource = df.areasource.tolist()
areasource_domain = list(get_domain('areasource', 'pluto_domains.json').keys())

In [65]:
# in domain not in pluto:
[i for i in areasource_domain if i not in unique_areasource]

[]

In [66]:
# in pluto not in domain:
[i for i in unique_areasource if i not in areasource_domain]

[None, None]

### Extension

In [67]:
sql = f'''
    WITH newcount as (
    SELECT ext, count(*)
    FROM {pluto}
    GROUP BY ext),
    prevcount as (
    SELECT ext, count(*)
    FROM {plutoprev}
    GROUP BY ext)
    SELECT a.ext, b.ext as extprev, a.count as countnew, b.count as countprev, a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.ext = b.ext
    ORDER BY a.ext;
'''

In [68]:
df = pd.read_sql(sql=sql, con=con)

In [69]:
df

,ext,extprev,countnew,countprev,diff
0,E,E,48478.0,48335.0,143.0
1,EG,EG,36790.0,36709.0,81.0
2,G,G,264848.0,264413.0,435.0
3,N,N,452422.0,451333.0,1089.0
4,None,None,57173.0,NaN,NaN
5,None,None,NaN,58669.0,NaN


In [70]:
#Check extension domain
unique_extension = df.ext.tolist()
extension_domain = list(get_domain('ext', 'pluto_domains.json').keys())

In [71]:
# in domain not in pluto:
[i for i in extension_domain if i not in unique_extension]

[]

In [72]:
# in pluto not in domain:
[i for i in unique_extension if i not in extension_domain]

['N', None, None]

### Proxy code

In [73]:
sql = f'''
    WITH newcount as (
    SELECT proxcode, count(*)
    FROM {pluto}
    GROUP BY proxcode),
    prevcount as (
    SELECT proxcode, count(*)
    FROM {plutoprev}
    GROUP BY proxcode)
    SELECT a.proxcode, b.proxcode as proxcodeprev, a.count as countnew, b.count as countprev, a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.proxcode = b.proxcode
    ORDER BY a.proxcode;
'''

In [ ]:
df = pd.read_sql(sql=sql, con=con)

In [ ]:
df

In [ ]:
#Check proxcode domain
unique_proxcode = df.proxcode.tolist()
proxcode_domain = list(get_domain('proxcode', 'pluto_domains.json').keys())

In [ ]:
# in domain not in pluto:
[i for i in proxcode_domain if i not in unique_proxcode]

In [ ]:
# in pluto not in domain:
[i for i in unique_proxcode if i not in proxcode_domain]

### Lot type

In [ ]:
sql = f'''
    WITH newcount as (
    SELECT lottype, count(*)
    FROM {pluto}
    GROUP BY lottype),
    prevcount as (
    SELECT lottype, count(*)
    FROM {plutoprev}
    GROUP BY lottype)
    SELECT a.lottype, b.lottype as lottypeprev, a.count as countnew, b.count as countprev, a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.lottype = b.lottype
    ORDER BY a.lottype;
'''

In [ ]:
df = pd.read_sql(sql=sql, con=con)

In [ ]:
df

In [ ]:
#Check lottype domain
unique_lottype = df.lottype.tolist()
lottype_domain = list(get_domain('lottype', 'pluto_domains.json').keys())

In [ ]:
# in domain not in pluto:
[i for i in lottype_domain if i not in unique_lottype]

In [ ]:
# in pluto not in domain:
[i for i in unique_lottype if i not in lottype_domain]

### Basement code

In [ ]:
sql = f'''
    WITH newcount as (
    SELECT bsmtcode, count(*)
    FROM {pluto}
    GROUP BY bsmtcode),
    prevcount as (
    SELECT bsmtcode, count(*)
    FROM {plutoprev}
    GROUP BY bsmtcode)
    SELECT a.bsmtcode, b.bsmtcode as bsmtcodeprev, a.count as countnew, b.count as countprev, a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.bsmtcode = b.bsmtcode
    ORDER BY a.bsmtcode;
'''

In [ ]:
df = pd.read_sql(sql=sql, con=con)

In [ ]:
df

In [ ]:
#Check bsmtcode domain
unique_bsmtcode = df.bsmtcode.tolist()
bsmtcode_domain = list(get_domain('bsmtcode', 'pluto_domains.json').keys())

In [ ]:
# in domain not in pluto:
[i for i in bsmtcode_domain if i not in unique_bsmtcode]

In [ ]:
# in pluto not in domain:
[i for i in unique_bsmtcode if i not in bsmtcode_domain]

### PLUTO map id

In [ ]:
sql = f'''
    WITH newcount as (
    SELECT plutomapid, count(*)
    FROM {pluto}
    GROUP BY plutomapid),
    prevcount as (
    SELECT plutomapid, count(*)
    FROM {plutoprev}
    GROUP BY plutomapid)
    SELECT a.plutomapid, b.plutomapid as plutomapidprev, a.count as countnew, b.count as countprev, a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.plutomapid = b.plutomapid
    ORDER BY a.plutomapid;
'''

In [ ]:
df = pd.read_sql(sql=sql, con=con)

In [ ]:
df

In [ ]:
#Check plutomapid domain
unique_plutomapid = df.plutomapid.tolist()
plutomapid_domain = list(get_domain('plutomapid', 'pluto_domains.json').keys())

In [ ]:
# in domain not in pluto:
[i for i in plutomapid_domain if i not in unique_plutomapid]

In [ ]:
# in pluto not in domain:
[i for i in unique_plutomapid if i not in plutomapid_domain]

#### done